# 3. Lasso Auto + All Features 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Lasso
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
df_orig = pd.read_csv('../../../full_dataset_unscaled.csv')
df_orig = df_orig.rename(columns={"datetime":"year-month"})
df_orig['net_job_rate']=df_orig['job_creation_rate']-df_orig['job_destruction_rate']
df_orig['apartment_for_rent_searches'] = df_orig[['apartment for rent','studio for rent','1 bedroom for rent','3 bedroom for rent']].sum(axis=1)
df_orig['multifamily_for_rent_searches'] = df_orig[['townhomes for rent','townhouse for rent','house for rent','duplex apartments for rent','condos for rent']].sum(axis=1)
df_orig['gun_searches'] = df_orig[['gun range', 'gun control', 'gun violence']].sum(axis=1)
df_orig['zri'] = np.log(df_orig['zri'])

In [3]:
gen_cols = ['zip',
 'City',
 'State',
 'Metro',
 'CountyName',
 'zri',
 'year',
 'month',
 'year-month']
acs_cols = ['percent_white',
 'percent_black',
 'percent_asian',
 'percent_hispanic',
 'percent_native_am',
 'percent_other_race',
 'percent_0_17',
 'percent_18_39',
 'percent_40_64',
 'percent_65+',
 'percent_rental_units_vacant',
 'percent_rental_units_occupied',
 'percent_graduate_deg',
 'percent_bachelors',
 'percent_associates',
 'percent_highschool',
 'percent_less_highschool',
 'percent_commute_public_transport',
 'percent_commute_less_30',
 'percent_buildings_less_10_units',
 'percent_buildings_10_19_units',
 'percent_buildings_20_49_units',
 'percent_buildings_50+_units',
 'percent_commute_30_to_59',
 'percent_commute_60_to_89',
 'percent_commute_90_more',
 'percent_new_city',
 'percent_new_unit',
 'percent_units_owner_occupied',
 'median_building_age',
 'income_per_capita',
 'poverty_rate',
 'total_pop',
 'percent_workforce_unemployed',
 'percent_work_from_home',
 'median_age',
 'percent_female',
 'gini_index',
 'percent_not_us_citizen']
acs_cols_remove=['percent_other_race','percent_40_64','percent_0_17','percent_18_39',
       'percent_65+','percent_rental_units_vacant','percent_not_us_citizen','percent_less_highschool', 'percent_buildings_less_10_units',
 'percent_commute_30_to_59',
 'percent_commute_60_to_89',
 'percent_commute_90_more', 'percent_commute_less_30','percent_graduate_deg',
                'percent_female','gini_index','percent_hispanic','percent_black','percent_bachelors',
                 'percent_asian','percent_new_city','percent_new_unit']
acs_cols_keep=list(set(acs_cols) - set(acs_cols_remove))
bikeshare_cols = ['bs_total_stations',
 'bs_total_systems',
 'has_bike_sharing']
bikeshare_cols_remove=['has_bike_sharing','bs_total_systems']
bikeshare_cols_keep=list(set(bikeshare_cols) - set(bikeshare_cols_remove))
trends_cols = ['gun range',
 'gun control',
 'gun violence',
 'job opportunities',
 'unemployment',
 'retirement',
 'layoff',
 'lgbt',
 'same sex marriage',
 'they',
 'pronouns',
 'black lives matter',
 'political correctness',
 'make america great again',
 'euthanasia',
 'getaway',
 'places to go',
 'flight tickets',
 'twitter',
 'hashtag',
 'fake news',
 'hurricane',
 'wildfire',
 'flood',
 'fire',
 "trader joe's",
 'whole foods',
 'lululemon',
 'thrift',
 'condos for rent',
 'duplex apartments for rent',
 'townhomes for rent',
 'townhouses for rent',
 'home for rent',
 'house for rent',
 'townhome for rent',
 'townhouse for rent',
 'apartment for rent',
 'studio for rent',
 '1 bedroom for rent',
 '3 bedroom for rent',
 'starbucks',
  'apartment_for_rent_searches',
  'multifamily_for_rent_searches',
  'gun_searches']
trends_cols_remove=['they','apartment for rent','studio for rent','1 bedroom for rent',
                    '3 bedroom for rent', 'townhome for rent','townhouse for rent','townhomes for rent',
                    'townhouses for rent','house for rent','home for rent','duplex apartments for rent','condos for rent',
                   'gun range', 'gun control', 'gun violence','homes for rent']
trends_cols_keep=list(set(trends_cols) - set(trends_cols_remove))
economic_cols = ['total_firms',
 'job_creation_rate',
 'job_destruction_rate',
 'startup_firms','state_local_perc', 'net_job_rate']
economic_cols_remove=['total_firms', 'job_creation_rate','job_destruction_rate',]
economic_cols_keep=list(set(economic_cols) - set(economic_cols_remove))

In [4]:
for zipcode in df_orig['zip'].unique():
    globals()[f"scaler_{zipcode}"]=StandardScaler(copy=False)
    df_filtered=df_orig[df_orig['zip']==zipcode]
    df_filtered_train = df_filtered[df_filtered['year']<2019]
    globals()[f"scaler_{zipcode}"].fit(df_filtered_train[['zri']])
    df_orig.loc[df_orig['zip']==zipcode,'zri']=globals()[f"scaler_{zipcode}"].transform(df_filtered[['zri']])

In [5]:
df_orig = df_orig[gen_cols + acs_cols_keep + bikeshare_cols_keep + economic_cols_keep + trends_cols_keep]

In [6]:
scale_columns = ['percent_buildings_50+_units', 'percent_associates',
       'percent_rental_units_occupied', 'percent_white', 'percent_highschool',
       'percent_work_from_home', 'percent_buildings_20_49_units',
       'median_building_age', 'median_age', 'percent_commute_public_transport',
       'percent_buildings_10_19_units', 'income_per_capita',
       'percent_native_am', 'percent_workforce_unemployed', 'poverty_rate',
       'percent_units_owner_occupied', 'total_pop', 'bs_total_stations',
       'startup_firms', 'state_local_perc', 'net_job_rate', 'gun_searches',
       'wildfire', 'fire', 'lgbt', 'political correctness', 'lululemon',
       'make america great again', 'same sex marriage', 'job opportunities',
       'retirement', 'black lives matter', 'flight tickets', 'pronouns',
       'trader joe\'s', 'fake news', 'hurricane', 'flood', 'whole foods',
       'twitter', 'thrift', 'hashtag', 'apartment_for_rent_searches', 'layoff',
       'starbucks', 'getaway', 'places to go', 'unemployment', 'euthanasia',
       'multifamily_for_rent_searches']

In [7]:
for zipcode in df_orig['zip'].unique():
    globals()[f"scaler_features_{zipcode}"]=StandardScaler(copy=False)
    df_filtered=df_orig[df_orig['zip']==zipcode]
    df_filtered_train = df_filtered[df_filtered['year']<2019]
    globals()[f"scaler_features_{zipcode}"].fit(df_filtered_train[scale_columns])
    df_orig.loc[df_orig['zip']==zipcode,scale_columns]=globals()[f"scaler_features_{zipcode}"].transform(df_filtered[scale_columns])

In [8]:
df_zri_all = df_orig

In [9]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

def laggenerator_diff(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+1]
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+1}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+1)
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+1}']=0
    return df

def laggenerator_diff12(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i+11]
    df.loc[:,f'{colname}_diff_lag{i}_lag{i+11}']=df.loc[:,f'{colname}'].shift(i)-df.loc[:,f'{colname}'].shift(i+11)
    df.loc[df['year-month'].isin(timelist),f'{colname}_diff_lag{i}_lag{i+11}']=0
    return df

#ZRI 
def lag_gen(df):
    for i in range(1,12):
        df=laggenerator(i, 'zri', df)
    df=laggenerator_diff12(1, 'zri', df)
    return df

In [10]:
#ZRI
df_zri_all = lag_gen(df_zri_all)

#ACS
for col in acs_cols_keep:
    df_zri_all = laggenerator_diff12(1,col,df_zri_all)
    df_zri_all = laggenerator(1,col,df_zri_all)
    df_zri_all = laggenerator(6,col,df_zri_all)
    df_zri_all = laggenerator(12,col,df_zri_all)
    
#BIKESHARE AND ECONOMIC 
for col in bikeshare_cols_keep + economic_cols_keep:
    df_zri_all = laggenerator(1, col, df_zri_all)
    df_zri_all = laggenerator_diff12(1, col, df_zri_all)  
    
#TRENDS 
for col in trends_cols_keep:
    for i in range(1,4):
        df_zri_all=laggenerator(i, col, df_zri_all)

In [11]:
df = df_zri_all[df_zri_all['year-month']>='2015-04-01']
train = df[(df['year'] < 2019) & (df['year']>=2015)]
test = df[df['year']==2019]

train = train.sort_values(by='year-month',ascending=True)
test = test.sort_values(by='year-month',ascending=True)

save_train = train.copy()
save_test = test.copy()



In [12]:
train = train.drop(['zip','City','State','Metro','CountyName','year','month','year-month'],axis=1)
train_y = train['zri']
train_X = train.drop(['zri'],axis=1)

test_y = test['zri']
test_X = test

In [16]:
#Lasso Grid Search
fold=TimeSeriesSplit(n_splits=5)
lasso = Lasso()
grid = dict()
grid['alpha'] = [0.001,0.01,0.05,0.1,0.5,1,10]
lasso_grid = GridSearchCV(lasso, grid, cv=fold, n_jobs=-1).fit(train_X,train_y)
#lasso_best = lasso_grid.best_estimator_

In [39]:
lasso_best = Lasso(alpha=0.01)

In [40]:
lasso_best.fit(train_X, train_y)

Lasso(alpha=0.01)

In [41]:
predictor_table=df.copy()
X_test = test_X.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    
    #run prediction for one month
    X_test=X_test[X_test['year-month']==month]
    X_test=X_test.drop(['zip','year','month','City','State','Metro','CountyName','year-month','zri'],axis=1)
    val=lasso_best.predict(X_test)

    #write current month prediction into predictor_table

    predictor_table.loc[predictor_table['year-month']==month,'zri']=val
        
    predictor_table=lag_gen(predictor_table)

    X_test=predictor_table[predictor_table['year']==2019]

scaled_predictions_y = X_test['zri']
temp = pd.concat([save_test['zip'],test_y],axis=1)
temp.reset_index(drop=True, inplace=True)
scaled_predictions_y.reset_index(drop=True, inplace=True)
rstable = pd.concat([temp,scaled_predictions_y],axis=1)
rstable.columns = ['zip','zri_test','zri_predicted']

for zipcode in rstable['zip'].unique():
    rstable_filtered=rstable[rstable['zip']==zipcode]
    rstable.loc[rstable['zip']==zipcode,'zri_test']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_test']])
    rstable.loc[rstable['zip']==zipcode,'zri_predicted']=globals()[f"scaler_{zipcode}"].inverse_transform(rstable_filtered[['zri_predicted']])  
rstable.loc[:,'zri_test'] = np.exp(rstable.loc[:,'zri_test'])
rstable.loc[:,'zri_predicted'] = np.exp(rstable.loc[:,'zri_predicted'])
rstable

,zip,zri_test,zri_predicted
0,1013,1099.0,1104.739851
1,14850,1545.0,1597.069313
2,6114,1139.0,1092.595199
3,90804,1957.0,1808.995912
4,59601,845.4,843.381935
...,...,...,...
15607,14217,901.0,923.533427
15608,60611,2138.0,2240.269399
15609,93307,883.0,899.062315
15610,10019,3619.0,3745.236626


In [42]:
#RESULTS 
r2 = r2_score(rstable['zri_test'],rstable['zri_predicted'])
rmse = sqrt(mean_squared_error(rstable['zri_test'],rstable['zri_predicted']))
print(f'R2: {r2*100}')
print(f'RMSE: {rmse}')

R2: 96.78959391649265
RMSE: 119.20767979158444


In [43]:
importance = np.abs(lasso_best.coef_)
coef = lasso_best.coef_
feature_names = train_X.columns
feature_importances= pd.DataFrame([feature_names,importance,coef]).T
feature_importances.columns = ['feature_names','coef_abs','coef']

feature_importances = feature_importances.sort_values(by='coef_abs',ascending=False)
feature_importances['Parent_feature']=feature_importances['feature_names'].apply(lambda x: x[0:x.find('_lag')] if x.find('_lag')!=-1 else x)
feature_importances['Parent_feature']=feature_importances['Parent_feature'].apply(lambda x: x[0:x.find('_diff')] if x.find('_diff')!=-1 else x)

In [44]:
feature_importances

,feature_names,coef_abs,coef,Parent_feature
50,zri_lag1,1.029261,1.029261,zri
52,zri_lag3,0.094326,-0.094326,zri
53,zri_lag4,0.014765,-0.014765,zri
224,fake news_lag3,0.006512,0.006512,fake news
183,multifamily_for_rent_searches_lag1,0.004906,0.004906,multifamily_for_rent_searches
44,places to go,0.004255,0.004255,places to go
184,multifamily_for_rent_searches_lag2,0.003555,0.003555,multifamily_for_rent_searches
207,places to go_lag1,0.003092,0.003092,places to go
186,lululemon_lag1,0.00169,-0.00169,lululemon
37,lululemon,0.001012,-0.001012,lululemon


In [45]:
feats_agg = feature_importances[['coef_abs','Parent_feature']]
feats_agg['coef_abs'] = feats_agg['coef_abs'].astype('float')
feats_agg_abs = feats_agg.groupby('Parent_feature').agg('mean').sort_values('coef_abs',ascending=False)
feats_agg_abs

<ipython-input-45-0836822478da>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats_agg['coef_abs'] = feats_agg['coef_abs'].astype('float')


,coef_abs
Parent_feature,
zri,0.094863
multifamily_for_rent_searches,0.002115
places to go,0.001935
fake news,0.001628
lululemon,0.000675
retirement,0.000232
percent_rental_units_occupied,0.000000
percent_units_owner_occupied,0.000000
percent_white,0.000000


In [46]:
title = 'FINAL_3_Auto_All'
fi = title+'_FI'
fi2 = title+'_FI_Agg_sign'
# rstable.to_csv(f'../../../zillow_orientation/Residuals/{title}.csv')
# feature_importances.to_csv(f'../../../zillow_orientation/Residuals/{fi}.csv')
# feats_agg_abs.to_csv(f'../../../zillow_orientation/Residuals/{fi2}.csv')

In [47]:
# R2: 96.78959391649265
# RMSE: 119.20767979158444
# alpha: 0.01

In [48]:
rstable

,zip,zri_test,zri_predicted
0,1013,1099.0,1104.739851
1,14850,1545.0,1597.069313
2,6114,1139.0,1092.595199
3,90804,1957.0,1808.995912
4,59601,845.4,843.381935
...,...,...,...
15607,14217,901.0,923.533427
15608,60611,2138.0,2240.269399
15609,93307,883.0,899.062315
15610,10019,3619.0,3745.236626


In [50]:
rstable['residual']=rstable['zri_predicted']-rstable['zri_test']
rstable['residual_squared']=rstable['residual'].map(lambda x: x**2)
rstable

,zip,zri_test,zri_predicted,residual,residual_squared
0,1013,1099.0,1104.739851,5.739851,32.945891
1,14850,1545.0,1597.069313,52.069313,2711.213399
2,6114,1139.0,1092.595199,-46.404801,2153.405570
3,90804,1957.0,1808.995912,-148.004088,21905.210126
4,59601,845.4,843.381935,-2.018065,4.072586
...,...,...,...,...,...
15607,14217,901.0,923.533427,22.533427,507.755310
15608,60611,2138.0,2240.269399,102.269399,10459.029886
15609,93307,883.0,899.062315,16.062315,257.997975
15610,10019,3619.0,3745.236626,126.236626,15935.685619


In [51]:
rmse_table=rstable.groupby('zip')[['residual_squared']].agg(['sum','count'])
rmse_table['residual_avgd']=rmse_table['residual_squared']['sum']/rmse_table['residual_squared']['count']
rmse_table['residual_avgd']=rmse_table['residual_avgd'].apply(lambda x: x**0.5)
rmse_table['residual_avgd'].mean()

97.04144335035538

In [53]:
rmse_table

residual_squared       residual_avgd
                   sum count              
zip                                       
1013      17740.160245    12     38.449274
1020      56314.963109    12     68.504844
1040      35119.644684    12     54.098402
1085      33934.647547    12     53.177884
1104      29247.880239    12     49.369255
...                ...   ...           ...
99207    270990.441717    12    150.274871
99501     63730.771843    12     72.875906
99504    129130.050462    12    103.734457
99508     68081.760726    12     75.322507
99654     51232.479895    12     65.340442

[1301 rows x 3 columns]

In [52]:
#RESULTS 
r2 = r2_score(rstable['zri_test'],rstable['zri_predicted'])
rmse = sqrt(mean_squared_error(rstable['zri_test'],rstable['zri_predicted']))
print(f'R2: {r2*100}')
print(f'RMSE: {rmse}')

R2: 96.78959391649265
RMSE: 119.20767979158444


In [54]:
# rstable.to_csv(f'../../../zillow_orientation/Residuals/{title}_rstable.csv')
# rmse_table.to_csv(f'../../../zillow_orientation/Residuals/{title}_rmse.csv')